<a href="https://colab.research.google.com/github/mallow5359/Mallow-s-Toolbox/blob/main/%E7%9C%81%E4%BB%BD%E6%95%B0%E6%8D%AE%E7%BB%9F%E8%AE%A1/%E7%9C%81%E4%BB%BD%E6%95%B0%E6%8D%AE%E7%BB%9F%E8%AE%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import xlrd
from openpyxl import Workbook
from collections import Counter
import re

# 挂载谷歌云盘
from google.colab import drive
drive.mount('/content/drive/')

# 打开文件
ExcelFile = xlrd.open_workbook('/content/drive/MyDrive/Colab/省份数据统计/全国地方登记社会组织-至2020年8月16日.xlsx')
sheet = ExcelFile.sheet_by_index(0)
CodeFile = xlrd.open_workbook('/content/drive/MyDrive/Colab/省份数据统计/行政区划代码2021.xlsx')
CodeSheet = CodeFile.sheet_by_index(0)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# 读取社会统一信用代码
name = sheet.col_values(0)
code = sheet.col_values(1)

# 删除第一个元素（即"统一社会信用代码"这个标题）
del code[0]

# 读取区划代码
code2021_code = CodeSheet.col_values(0)
code2021_city = CodeSheet.col_values(1)
assert len(code2021_code) == len(code2021_city)

# 区划代码原本是floatfloat，转换成int再转换成str
for i in range(len(code2021_code)):
  code2021_code[i] = str(int(code2021_code[i]))

# 提取省份和城市的区划代码并汇总成字典
code_province = {}
code_city = {}
for i in range(len(code2021_code)):
  if code2021_code[i][2:6] == '0000':
    code_province.setdefault(code2021_code[i][0:2], code2021_city[i])
  if code2021_code[i][4:6] == '00':
    code_city.setdefault(code2021_code[i][0:4], code2021_city[i])
  code_province.update({'10':'国家直属单位', '66':'新疆建设兵团'})

counter_province = Counter()
counter_city = Counter()

invalid_data_count = 0
find_chinese = re.compile(r'[\u4e00-\u9fa5]+')
find_point = re.compile(r'[/.]+')
for i in code:
  if  find_point.findall(str(i)) != []:
    i = str(int(i))
  else:
    i = str(i)
  if find_chinese.findall(i) == []:
    counter_province[i[2:4]] += 1
    counter_city[i[2:6]] += 1
  else:
    invalid_data_count += 1
assert sum(counter_province.values()) == sum(counter_city.values()) == len(code) - invalid_data_count

for i in code_province.keys(): 
  print(code_province.get(i), counter_province.get(i[0:2]))

for i in code_city.keys():
  print(code_city.get(i), counter_city.get(i[0:4]))

In [ ]:
excel_output = Workbook()
sheet1 = excel_output.create_sheet('Sheet1', 0)
sheet1.cell(row=1, column=1).value = '省级分类统计'
count = 1
for i in code_province.keys():
  count += 1
  sheet1.cell(row=count, column=1).value = code_province.get(i)
  sheet1.cell(row=count, column=2).value = counter_province.get(i[0:2])
sheet1.cell(row=len(code_province.keys())+2, column=1).value = '市级分类统计（包含省级直属）'
count += 1
for i in code_city.keys():
  count += 1
  sheet1.cell(row=count, column=1).value = code_city.get(i)
  sheet1.cell(row=count, column=2).value = counter_city.get(i[0:4])
excel_output.save('/content/drive/MyDrive/Colab/省份数据统计/统计结果.xlsx')

In [ ]:
find_point = re.compile(r'[/.]+')

count = 0
for i in code:
  count += 1
  if  find_point.findall(str(i)) != []:
    i = str(int(i))
  else:
    i = str(i)
  '''flag = 0
  for j in code_province.keys():
    if i[2:4] == j:
      flag = 1
  if flag == 0:
    print('province_error:', count+ 1, i, name[count - 1])'''
  flag = 0
  for k in code_city.keys():
    if i[2:6] == k or i[2:4] == '11' or i[2:4] == '12' or i[2:4] == '31' or i[2:4] == '50' or i[2:4] == '10' or i[2:4] == '66':
      flag = 1
  if flag == 0:
    print('city_error:', count + 1, i, name[count])
